In [16]:
import concurrent.futures
import os
import re
import subprocess
import sys
import tempfile
from pathlib import Path

import jax
import kmbio
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from elaspic2.plugins.alphafold import AlphaFold, AlphaFoldAnalyzeError, AlphaFoldBuildError
from jax.lib import xla_bridge
from kmbio import PDB
from tqdm.notebook import tqdm

In [3]:
NOTEBOOK_DIR = Path("31_run_alphafold_wt").resolve()
NOTEBOOK_DIR.mkdir(exist_ok=True)

NOTEBOOK_DIR

PosixPath('/lustre07/scratch/sunyun/workspace/elaspic2-cagi6/notebooks/31_run_alphafold_wt')

In [8]:
if (slurm_tmpdir := os.getenv("SLURM_TMPDIR")) is not None:
    os.environ["TMPDIR"] = slurm_tmpdir

print(tempfile.gettempdir())

/tmp


In [9]:
DATASET_NAME = os.getenv("DATASET_NAME")
DATASET_PATH = os.getenv("DATASET_PATH")
TASK_ID = os.getenv("SLURM_ARRAY_TASK_ID")
TASK_COUNT = os.getenv("ORIGINAL_ARRAY_TASK_COUNT") or os.getenv("SLURM_ARRAY_TASK_COUNT")
ARRAY_TASK_OFFSET = int(os.getenv("ARRAY_TASK_OFFSET", "0"))

TASK_ID = (int(TASK_ID) + ARRAY_TASK_OFFSET) if TASK_ID is not None else None
TASK_COUNT = int(TASK_COUNT) if TASK_COUNT is not None else None 

DATASET_NAME, DATASET_PATH, TASK_ID, TASK_COUNT, ARRAY_TASK_OFFSET

(None, None, None, None, 0)

In [10]:
DEBUG = TASK_ID is None

if DEBUG:
    DATASET_NAME = "humsavar"
    DATASET_PATH = str(
        NOTEBOOK_DIR.parent.joinpath("30_humsavar", "humsavar-gby-protein-waln.parquet")
    )
    TASK_ID = 10000
    TASK_COUNT = 12557
else:
    assert DATASET_NAME is not None
    assert DATASET_PATH is not None 
    DATASET_PATH = Path(DATASET_PATH).expanduser().resolve()
    assert TASK_ID is not None
    assert TASK_COUNT is not None

DATASET_NAME, DATASET_PATH, TASK_ID, TASK_COUNT

('humsavar',
 '/lustre07/scratch/sunyun/workspace/elaspic2-cagi6/notebooks/30_humsavar/humsavar-gby-protein-waln.parquet',
 10000,
 12557)

In [13]:
device = xla_bridge.get_backend().platform

device

2021-11-03 19:10:09.218277: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64:/.singularity.d/libs
2021-11-03 19:10:09.218302: W external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


'cpu'

In [14]:
output_file = NOTEBOOK_DIR.joinpath(DATASET_NAME, f"shard-{TASK_ID}-of-{TASK_COUNT}.parquet")
output_file.parent.mkdir(exist_ok=True)

output_file

PosixPath('/lustre07/scratch/sunyun/workspace/elaspic2-cagi6/notebooks/31_run_alphafold_wt/humsavar/shard-10000-of-12557.parquet')

In [15]:
if output_file.is_file():
    raise Exception("Already finished!")

## Workspace

### Initialize model

In [17]:
AlphaFold.load_model(device=device)

## Load data

In [21]:
pfile = pq.ParquetFile(DATASET_PATH)
pfile.num_row_groups

12557

In [22]:
assert TASK_COUNT == pfile.num_row_groups

In [32]:
pfile.read_row_group(0).to_pandas(integer_object_nulls=True)

,protein_id,mutation,effect,sequence,structure,alignment
0,A0A0C5B5G6,[K14Q],[US],MRWQEMGYIFYPRKLR,HEADER ...,"[>101\n, MRWQEMGYIFYPRKLR\n, >UniRef100_A0A0C5..."


In [33]:
input_df = pfile.read_row_group(TASK_ID - 1).to_pandas(integer_object_nulls=True)

In [36]:
display(input_df.head(1))
print(len(input_df))

,protein_id,mutation,effect,sequence,structure,alignment
0,Q13415,"[F89S, E127G, A372V, R441M, K456E, T466M, M816T]","[LP/P, LP/P, LB/B, LB/B, LB/B, LB/B, LB/B]",MAHYPTRLKTRKTYSWVGRPLLDRKLHYQTYREMCVKTEGCSTEIH...,HEADER ...,"[>101\n, MAHYPTRLKTRKTYSWVGRPLLDRKLHYQTYREMCVK..."


1


In [64]:
getattr(tup, field)

0    HEADER                                        ...
Name: structure, dtype: object

In [69]:
# tup = next(input_df.itertuples(index=False))
# for field in tup._fields: 
#     print(field)
#     print(getattr(tup, field))
#     print()

In [37]:
protein_id_column = None

for col in ["protein_id", "uniprot_id"]:
    if col in input_df: 
        protein_id_column = col

assert protein_id_column is not None
protein_id_column

'protein_id'

In [51]:
# Get "mutation", "effect" 
tup = next(input_df.itertuples(index=False))
iterable_fields = []
for field in tup._fields: 
    if field in [protein_id_column]:
        continue
    try: 
        
        if len(getattr(tup, field)) == len(tup.mutation):
            iterable_fields.append(field)
    except TypeError: 
        pass

iterable_fields

['mutation', 'effect']

In [55]:
# Only get the first three mutations 
if DEBUG:
    for field in iterable_fields: 
        input_df[field] = input_df[field].str[:3]
    input_df

### Score mutations

In [ ]:
def validate_mutation(mutation):
    aa = "GVALICMFWPDESTYQNKRH"
    if re.search(f"^[{aa}][1-9]+[0-9]*[{aa}]$", mutation) is None: 
        print(f"Skipping mutation {mutation} because it appears to be malformed.")
        return False
    
    if mutation[0] == mutation[-1]:
        print(
            f"Skipping mutation {mutation} because the wildtype and mutant residues are the same."
        )
        return False
    
    return True

In [61]:
def predictions_to_embeddings(predictions):
    return {
        "plddt": pa.array([predictions["plddt"]]),
        "max_predicted_aligned_error": pa.array(
            [predictions["max_predicted_aligned_error"].item()]
        ),
        "ptm": pa.array([predictions["ptm"].item()]),
        #
        "experimentally_resolved": pa.array(
            [predictions["experimentally_resolved"]["logits"].to_py().tolist()]
        ),
        "predicted_lddt": pa.array([predictions["predicted_lddt"]["logits"].to_py().tolist()]),
        #
        "msa_first_row": pa.array(
            [predictions["representations"]["msa_first_row"].to_py().tolist()]
        ),
        "single": pa.array([predictions["representations"]["single"].to_py().tolist()]),
        "structure_module": pa.array(
            [predictions["representations"]["structure_module"].to_py().tolist()]
        ),
        # Pairwise metrics
        "distogram": pa.array([data.predictions["distogram"]["logits"].to_py().tolist()]),
        "distogram_bin_edges": pa.array(
            [data.predictions["distogram"]["bin_edges"].to_py().tolist()]
        ),
        "masked_msa": pa.array([data.predictions["masked_msa"]["logits"].to_py().tolist()]),
        "predicted_aligned_error": pa.array([data.predictions["predicted_aligned_error"].tolist()]),
        "aligned_confidence_probs": pa.array(
            [data.predictions["aligned_confidence_probs"].tolist()]
        ),
        "msa": pa.array([data.predictions["representations"]["msa"].to_py().tolist()]),
        "pair": pa.array([data.predictions["representations"]["pair"].to_py().tolist()]),
    }

In [62]:
assert len(input_df) == 1

for tup in tqdm(input_df.itertuples(index=False), total=len(input_df)):
    assert all([(len(getattr(tup, field)) == len(tup.mutation)) for field in iterable_fields])

    with tempfile.NamedTemporaryFile(suffix=".pdb") as tmp_file:
        with open(tmp_file.name, "wt") as fout:
            fout.write(tup.structure)
        data = AlphaFold.build(
            tup.sequence, ligand_sequence=None, msa=tup.alignment, structure_file=tmp_file.name
        )

    results = {"protein_id": pa.array([tup.protein_id])} | predictions_to_embeddings(
        data.predictions
    )

    del data

AttributeError: 'gemmi.Structure' object has no attribute 'make_mmcif_block'

In [ ]:
table = pa.Table.from_pydict(results)
del results

### Save results

In [63]:
if not DEBUG:
    pq.write_table(table, output_file)
else: 
    pq.write_table(table, Path(tempfile.gettempdir(), output_file.name))

NameError: name 'table' is not defined